In [15]:
import pandas as pd
import numpy as np
import pytz
import datetime
import os
pd.set_option('display.max_rows', None)

In [3]:
aph_zipcodes = list(map(str, [76574, 78605, 78610, 78612, 78613, 78615, 78616, 78617, 78620, 78621, 78626, 78628, 78634, 78640, 78641, 
                78642, 78644, 78645, 78652, 78653, 78654, 78660, 78663, 78664, 78665, 78669, 78681, 78701, 78702, 78703, 
                78704, 78705, 78712, 78717, 78719, 78721, 78722, 78723, 78724, 78725, 78726, 78727, 78728, 78729, 78730, 
                78731, 78732, 78733, 78734, 78735, 78736, 78737, 78738, 78739, 78741, 78742, 78744, 78745, 78746, 78747, 
                78748, 78749, 78750, 78751, 78752, 78753, 78754, 78756, 78757, 78758, 78759]))

In [8]:
racks = '''

C-ROOM-00102
C-ROOM-00103
C-ROOM-00104
C-ROOM-00105


'''

In [44]:
#download https://metabase.curativeinc.com/question/10328-appt-tk-box-cn-v4

with os.scandir() as it:
    possible_ts_files = list()
    for entry in it:
        if entry.name.startswith('query_result'):
            possible_ts_files.append(entry.name)
samples_csv = max(possible_ts_files, key = os.path.getmtime)
samples = pd.read_csv(samples_csv)

In [56]:
samples.columns

Index(['Result', 'Completed At', 'Testing Site ID', 'Facility ID', 'Box ID',
       'Box X Position', 'Box Y Position', 'Test Kits → Barcode',
       'Box → Barcode', 'Alinity Results → Cn', 'Alinity Results → Created At',
       'POSN', 'stability'],
      dtype='object')

In [79]:
samples.rename(columns = {'Alinity Results → Cn': 'cn'}, inplace = True)
samples = samples.loc[samples.Result == 'POSITIVE']
samples.loc[:, 'Completed At'] = samples['Completed At'].apply(pd.to_datetime)
samples.loc[:, 'stability'] = datetime.datetime.now(tz = pytz.FixedOffset(-480)) - samples['Completed At']
samples.loc[:, 'Alinity Results → Created At'] = samples['Completed At'].apply(pd.to_datetime)
samples = samples.sort_values('Alinity Results → Created At').drop_duplicates('Test Kits → Barcode')

In [80]:
for i in samples['Test Kits → Barcode']: print(i)

A572270746170
A532804311891
A956392753083
A951872995695
A125991329255
A862156672433
A243890360651
A477690142152
A946874015514
A967147008302
A822775665948
A841746328481
A661350375706
A287482894396
A489093023974
A740312093079
A649815755180
A665303629188
A757464247732
A342968780863
A892175304027
A668788482821
A274176731954
A584495589865
A445352827192
A959497808035
A784186468374
A712261548728
A856901361047
A917342657594
A651964216152
A420193245144
A179903917700
A550416941645
A690190591110
A603525825788
A383197893265
A161987183580
A887171479350
A966866761671
A965488585966
A192663661148
A879365846772
A919026435097
A440745160251
A405174154253
A815507394340
A944231851875
A232998803540
A345175630594
A999199392120
A649492112936
A627725541053
A965848718750
A574646440192
A889754910217
A199144967128
A201924481241
A324402153594
A743592368883
A106048539734
A435998550910
A904872136154
A281861069221
A344374599986
A784336010342
A805258039118
A155773711875
A617311937984
A350224676635
A862909803617
A80192

In [37]:
samples.loc[samples.stability < datetime.timedelta(hours = 136)].groupby('Box → Barcode').count()

,Result,Completed At,Testing Site ID,Facility ID,Box ID,Box X Position,Box Y Position,Test Kits → Barcode,Alinity Results → Cn,Alinity Results → Created At,POSN,stability
Box → Barcode,,,,,,,,,,,,
C-ROOM-00102,54,54,54,54,54,54,54,54,54,54,54,54
C-ROOM-00103,77,77,77,77,77,77,77,77,77,77,77,77
C-ROOM-00104,81,81,81,81,81,81,81,81,81,81,81,81
C-ROOM-00105,84,84,84,84,84,84,84,84,84,84,84,84


In [48]:
with os.scandir() as it:
    possible_phi_files = list()
    for entry in it:
        if entry.name.startswith('test_kits_possam_doh_phi__'):
            possible_phi_files.append(entry.name)
phi_csv = max(possible_phi_files, key = os.path.getmtime)

In [63]:
phi = pd.read_csv(phi_csv)
phi.fillna(value={'patients_zipcode':10000}, inplace = True)
phi.loc[:, 'patients_zipcode'] = phi.patients_zipcode.apply(lambda x: str(x)[:5])

In [64]:
samples.columns

Index(['Result', 'Completed At', 'Testing Site ID', 'Facility ID', 'Box ID',
       'Box X Position', 'Box Y Position', 'Test Kits → Barcode',
       'Box → Barcode', 'cn', 'Alinity Results → Created At', 'POSN',
       'stability'],
      dtype='object')

In [81]:
df = samples.set_index('Test Kits → Barcode')\
    .join(phi[['barcode_id', 'patients_zipcode']].set_index('barcode_id'))\
    .reset_index()

In [82]:
#ALL THE POSITIVES
df.loc[df.Result == 'POSITIVE']\
[['Test Kits → Barcode', 'Result', 'Box → Barcode', 'POSN']].sort_values(by=['Box → Barcode', 'POSN'])

,Test Kits → Barcode,Result,Box → Barcode,POSN
36,A383197893265,POSITIVE,C-ROOM-00102,B3
86,A469738308932,POSITIVE,C-ROOM-00102,B4
84,A482542109645,POSITIVE,C-ROOM-00102,B5
78,A179531959033,POSITIVE,C-ROOM-00102,C1
67,A155773711875,POSITIVE,C-ROOM-00102,C2
27,A712261548728,POSITIVE,C-ROOM-00102,C3
53,A965848718750,POSITIVE,C-ROOM-00102,C4
48,A232998803540,POSITIVE,C-ROOM-00102,C5
16,A649815755180,POSITIVE,C-ROOM-00102,C8
74,A252559531851,POSITIVE,C-ROOM-00102,D1


In [67]:
df.columns

Index(['Test Kits → Barcode', 'Result', 'Completed At', 'Testing Site ID',
       'Facility ID', 'Box ID', 'Box X Position', 'Box Y Position',
       'Box → Barcode', 'cn', 'Alinity Results → Created At', 'POSN',
       'stability', 'patients_zipcode'],
      dtype='object')

In [83]:
#APH SAMPLES
df.loc[(df['Result']=='POSITIVE')&(df['cn']<26)&(df['patients_zipcode'].isin(aph_zipcodes))]\
[['Test Kits → Barcode', 'Box → Barcode', 'cn', 'POSN']].sort_values(by=['Box → Barcode', 'POSN'])

,Test Kits → Barcode,Box → Barcode,cn,POSN
88,A458203963184,C-ROOM-00104,21.99,F2


In [84]:
#MONROVIA SAMPLES
df.loc[(df['Result']=='POSITIVE')&(df['cn'] < 26)]\
[['Test Kits → Barcode', 'Box → Barcode', 'cn', 'POSN']].sort_values(by=['Box → Barcode', 'POSN'])

,Test Kits → Barcode,Box → Barcode,cn,POSN
88,A458203963184,C-ROOM-00104,21.99,F2


In [78]:
#EMPLOYEE SAMPLES
df.loc[df['Facility ID'] == 10]\
[['Test Kits → Barcode', 'Box → Barcode', 'cn', 'POSN']].sort_values(by=['Box → Barcode', 'POSN'])

,Test Kits → Barcode,Box → Barcode,cn,POSN
194,A200887621570,C-ROOM-00102,38.22,E1
183,A199144967128,C-ROOM-00102,31.54,F5
102,A339273559776,C-ROOM-00102,-1.00,F7
281,A470765483558,C-ROOM-00102,-1.00,F8
30,A477690142152,C-ROOM-00105,40.10,A10
10,A287749940140,C-ROOM-00105,-1.00,C11
8,A844652191130,C-ROOM-00105,-1.00,E1
257,A219776907942,C-ROOM-00105,-1.00,E2
239,A232409314918,C-ROOM-00105,-1.00,E3
253,A864556359253,C-ROOM-00105,-1.00,E4


In [22]:
#STRONG NEW MEXICO SAMPLES
df.loc[(df['site_state'] == 'NM') & (df['cn'] < 26)]\
[['Test Kits → Barcode', 'Box → Barcode', 'cn', 'POSN']].sort_values(by=['Box → Barcode', 'POSN'])

,Test Kits → Barcode,Box → Barcode,cn,POSN
